# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770434617347                   -0.52    9.0         
  2   -2.771688085338       -2.90       -1.32    1.0    185ms
  3   -2.771714287510       -4.58       -2.49    1.0    190ms
  4   -2.771714636368       -6.46       -3.22    1.0    216ms
  5   -2.771714713571       -7.11       -3.89    2.0    196ms
  6   -2.771714715211       -8.79       -5.27    1.0    193ms
  7   -2.771714715249      -10.42       -5.56    2.0    207ms
  8   -2.771714715250      -12.60       -6.25    1.0    180ms
  9   -2.771714715250      -13.23       -7.29    2.0    224ms
 10   -2.771714715250      -14.45       -7.59    1.0    191ms
 11   -2.771714715250   +  -14.75       -8.55    1.0    184ms


-0.00013457317634768816

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770483328026                   -0.52    9.0         
  2   -2.771777998282       -2.89       -1.33    1.0    211ms
  3   -2.771801528967       -4.63       -2.42    1.0    172ms
  4   -2.771801991314       -6.34       -3.11    1.0    172ms
  5   -2.771802074263       -7.08       -4.31    2.0    245ms
  6   -2.771802074419       -9.81       -4.61    1.0    177ms
  7   -2.771802074474      -10.26       -5.63    1.0    179ms
  8   -2.771802074476      -11.70       -6.06    2.0    253ms
  9   -2.771802074476      -12.97       -6.75    1.0    187ms
 10   -2.771802074476      -14.18       -7.25    2.0    229ms
 11   -2.771802074476   +  -14.57       -8.12    1.0    196ms


0.01761222387533247

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457317634768816
Displaced dipole:  0.01761222387533247
Polarizability :   1.7746797051680159


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920948244e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553069902e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852752809938e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694607979854e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088790833767e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.328084704267e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.878866941385e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.878866941385e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.800932733497e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.147281666635e-10
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.662510701783e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.662510701783e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.6625092618030128e-12
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.